<a href="https://colab.research.google.com/github/audrb1999/DL-Project/blob/main/01.numpy_XOR_%EA%B5%AC%ED%98%84/05._loss%EA%B0%92_%EA%B5%AC%ED%95%98%EA%B8%B0%2C_%EC%97%AD%EC%A0%84%ED%8C%8C_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이전 코드들

In [4]:
import numpy as np

# 이전 코드들
class Layer:
    def __init__(self):# 모든 함수들이 공통으로 입출력을 조절
        self.input = None
        self.output = None
    
    def forward(self, x):# 모든 함수들이 공통으로 사용하는 순전파 
        self.input = x
        self.output = self._forward(x)
        return self.output

    def _forward(self, x):
        raise NotImplementedError

class StepFunction(Layer):
    def _forward(self, x):
        y = x > 0
        return y.astype(np.int)

class Perceptron(Layer):
    def __init__(self, weights, bias, activation=None):
        super().__init__() # 만약 상위 클래스 Layer가 호출되지 않아도 실행할 수 있게하는 것
        self.weights = weights
        self.bias = bias
        self.activation = activation

    def _forward(self, x): # 활성화 함수 유무를 위한 if 문
        a = np.matmul(x, self.weights) + self.bias
        if self.activation != None:
            y = self.activation.forward(a)
            return y
        return a

class Loss:
    def forward(self, y_true, y_pred):
        raise NotImplementedError
   
    def _forward(self, y_true, y_pred):
        raise NotImplementedError

class MSE(Loss):
    def forward(self, y_true, y_pred):# forward를 상속받아서 고쳐쓴 것
        mse = np.mean(np.square(y_true - y_pred)/2, axis = 0, keepdims = True)# 1랭크가 되지 않도록 차원 유지!
        return mse

class Sigmoid(Layer):
    def _forward(self, x):
        return 1 / (1+np.exp(-x))

##***Sigmoid를 사용해서 xor을 구하고 loss값을 구하자!***

In [5]:
class Model(Layer):
    def __init__(self):
        super().__init__()
        self.layers = []
        self.loss = None #loss값 초기화 이전 값이 있더라도 model()클래스 선언시 초기화됨!

    def add(self, layer):
        self.layers.append(layer)


    def _forward(self, x):
        a = self.layers[0].forward(x)# 2층으로 연결해주기 위해서 이 출력을 다음 레이어가 받음
        c = self.layers[1].forward(x)
        y = []
        for i in range(len(a)):    
            z = Perceptron(np.array([0.3, 0.3]), 0.1, StepFunction()).forward(np.array([a[i], c[i]]))
            y.append(z)
            
        return y

    def set_loss(self, loss):
        self.loss = loss

    def evaluate(self, x, y):
        if self.loss is None:
            return None
        y_pred = Sigmoid().forward(x)
        
        return self.loss.forward(y, y_pred)


In [6]:
model = Model()
model.set_loss(MSE())
model.evaluate(np.random.normal(size=(4, 2)), np.ones((4, 2)))

array([[0.13909149, 0.13393503]])

##***Layer sigmoid역전파 구현!***

In [7]:
class Layer:
    def backward(self, grad = 1):
        if self.input is None or self.output is None:
            return None
        return self._backward(grad)

    def _backward(self, grad):
        raise NotImplementedError

class Sigmoid(Layer):
    def _forward(self, x):
        self.result = 1 / (1+np.exp(-x))
        return self.result

    def _backward(self, x): 
        result = self.result
        return result * (1 - result)
